Load data

In [23]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import pickle
from collections import defaultdict

from torch.utils.data import Dataset, DataLoader
from Mmetrics import *

import LTR
import datautil
import permutationgraph
import DTR
import EEL
import PPG
import PL

ds2020, _ = datautil.load_data(2020, verbose=True)
ds2019, _ = datautil.load_data(2019, verbose=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
train: 4694 docs, 200 queries.
test: 4677 docs, 200 queries.
25 features
un-normalized train: 70.080336425764 <101.65279842486342>
un-normalized test: 75.07174581020573 <156.60887376760795>
normalized train: 5.180490685311132e-07 <0.7804636401064724>
normalized test: 1.3643793263153821e-06 <0.7887188357739844>
train: 2672 docs, 632 queries.
test: 4298 docs, 635 queries.
25 features
un-normalized train: 100.59542148518399 <112.00393006016057>
un-normalized test: 107.18769941395159 <118.13138269196044>
normalized train: 7.412363746142352e-08 <0.7544854415648509>
normalized test: -2.8577670556600825e-08 <0.7641385980060638>


Fit a LTR model (MSE)

Set `y_pred` using the trained model.

In [21]:
ltrmodel = LTR.MSE_model(layers=[ds2020.trfm.shape[1], 256, 256, 1], lr=0.001, optimizer=torch.optim.Adam, dropout=0.1)
ltrmodel.fit(ds2020, epochs=10, batch_size=100, verbose=True)
y_pred2020 = ltrmodel.predict(ds2020.tefm, ds2020.tedlr)

ltrmodel = LTR.MSE_model(layers=[ds2019.trfm.shape[1], 256, 256, 1], lr=0.001, optimizer=torch.optim.Adam, dropout=0.1)
ltrmodel.fit(ds2019, epochs=10, batch_size=100, verbose=True)
y_pred2019 = ltrmodel.predict(ds2019.tefm, ds2019.tedlr)

epoch 0 -> loss: 0.052073169499635696, train ndcg@10: 0.3503035268320601, ndcg@10: 0.3876295792750259
epoch 1 -> loss: 0.01319053117185831, train ndcg@10: 0.35274867528084786, ndcg@10: 0.4052124510311721
epoch 2 -> loss: 0.010403155349195004, train ndcg@10: 0.3584947545414017, ndcg@10: 0.3978479170063658
epoch 3 -> loss: 0.009867612272500992, train ndcg@10: 0.3557424574748109, ndcg@10: 0.382260882004556
epoch 4 -> loss: 0.009421426802873611, train ndcg@10: 0.3526311535996758, ndcg@10: 0.39615375900304395
epoch 5 -> loss: 0.008984922431409359, train ndcg@10: 0.35874285320080174, ndcg@10: 0.38391187753334166
epoch 6 -> loss: 0.008523018099367619, train ndcg@10: 0.35984124128292594, ndcg@10: 0.46096574867997964
epoch 7 -> loss: 0.008614919148385525, train ndcg@10: 0.3573424055911887, ndcg@10: 0.4503731047894409
epoch 8 -> loss: 0.008287439122796059, train ndcg@10: 0.36766968062782157, ndcg@10: 0.4341627580062225
epoch 9 -> loss: 0.008117688819766045, train ndcg@10: 0.365028847843463, ndcg

---

# Query Learner

Test `QueryLearner` class for one query.

In [42]:
qid = 6
sessions = 1
s, e = ds2020.tedlr[qid:qid+2]
y_pred = y_pred2020[s:e]
sorted_docs = y_pred.argsort()[::-1]
g = ds2020.teg[s:e]


# objective_ins = DTR.DTR(y_pred = y_pred, g = g, dlr = None, exposure = np.array([1./np.log2(2+i) for i in range(1,1000)]), method='query_ratio')

y, g, sorted_docs, dlr = EEL.copy_sessions(y=y_pred, g=g, sorted=sorted_docs, sessions=sessions)

print(sorted_docs)
print(dlr)
objective_ins = EEL.EEL(y_pred = y, g = g, dlr = dlr, grade_levels=5, exposure = np.array([1./np.log2(2+i) for i in range(1,1000)]))

n = y.shape[0]
# learner = PPG.Learner(0.5 * np.triu(np.ones((n,n)), 1), samples_cnt=16, 
#                         objective_ins=objective_ins, sorted_docs=sorted_docs, dlr=dlr, intra=g, inter=np.repeat(dlr[:-1], np.diff(dlr)))
# learner = permutationgraph.QueryLearner(objective_ins, sorted_docs = sorted_docs, intra = g)
learner = PL.Learner(logits=y, samples_cnt=256, objective_ins=objective_ins)
learner.fit(50, 0.1, True)

[ 8  5 13  7 10  1  4 11  3  2  9  6 12  0]
[ 0 14]
0.0826042630947744
0.08682031267421593
0.08590702204536822
0.08289674670647809
0.0874206447701937
0.08010504353095313
0.08261609206241645
0.08368211650508939
0.08202088588406788
0.07617233341554151
0.07833836443458368
0.07452696181565661
0.07006656245057362
0.0729837372274111
0.06729107335231523
0.07798259535663231
0.07026930501554844
0.06864318862194192
0.07015272699750653
0.06030687241143994
0.07544672715708406
0.05327554591569512
0.056206461897259495
0.06308401940169454
0.05588075656411354
0.05858725071140732
0.05954103304489855
0.04804845773549559
0.047117885960439664
0.04911209512435585
0.052674635873865325
0.05050824498729071
0.044940633239440475
0.04682959010532906
0.04249167682661406
0.04305780390828842
0.043866091185220044
0.039166828808258364
0.031518524881159055
0.038420130678958546
0.040553995642120755
0.04030061776370093
0.03625325445635416
0.029557779192607072
0.03189113737680096
0.030772360815770828
0.0281408748645133
0

In [ ]:
learner = PPG.Learner(0.5 * np.triu(np.ones((n,n)), 1), samples_cnt=256, 
                        objective_ins=objective_ins, sorted_docs=sorted_docs, dlr=dlr, intra=g, inter=np.repeat(dlr[:-1], np.diff(dlr)))
learner.fit(50, 0.1, True)

Test `QueryLearner` class for all queries, using `learn_all_query` function.

In [15]:

def learn_all_query(y_pred, g, dlr, exposure, epochs, lr, learner_cls, objective, objective_args=None):
    y_rerank = []
    sorted_docs = []
    min_vals = []
    
    # for qid in trange(dlr.shape[0] - 1, leave=False):
    for qid in range(dlr.shape[0] - 1):
        s, e = dlr[qid:qid+2]
        if objective == 'DTR':
            objective_ins = DTR.DTR(y_pred = y_pred[s:e], g = g[s:e], dlr = None, exposure = exposure, method='query_ratio')
        elif objective == 'EEL':
            objective_ins = EEL.EEL(y_pred = y_pred[s:e], g = g[s:e], dlr = np.array([0,e-s]), exposure = exposure, **objective_args)

        learner = learner_cls(objective_ins, sorted_docs = y_pred[s:e].argsort()[::-1], intra = g[s:e])
        vals = learner.fit(epochs, lr, verbose=False)

        scores = np.arange(len(learner.sorted_docs), 0, -1)
        y_rerank.append(scores[learner.sorted_docs])
        sorted_docs.append(learner.sorted_docs)
        
        vals = np.array(vals)
        min_vals.append(vals.min())

    # print(ndcg_dtr(exposure, lv, np.concatenate(y_rerank), dlr, g, query_counts))
    return np.concatenate(y_rerank), np.concatenate(sorted_docs), np.array(min_vals)



**TREC 2020**

In [22]:
objective='EEL'
objective_args = {'grade_levels':2}

exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.tedlr).max()+2)])
y_rerank2020, sorted2020, min_vals = learn_all_query(  y_pred2020, ds2020.teg, ds2020.tedlr, 
                                            exposure = exposure2020,
                                            epochs=20, lr=0.3, 
                                            learner_cls=permutationgraph.QueryLearner,
                                            objective=objective,
                                            objective_args = objective_args)
print(f'{len(min_vals[min_vals>0])} valid queries. --> average: {min_vals[min_vals>0].mean()}')

if objective == 'DTR':
    print(DTR.ndcg_dtr(exposure2020, ds2020.telv, y_rerank2020, ds2020.tedlr, ds2020.teg, ds2020.query_seq))
elif objective == 'EEL':
    eel = EEL.EEL(y_pred=ds2020.telv, g=ds2020.teg, dlr=ds2020.tedlr, exposure=exposure2020, grade_levels=2)
    print(eel.eval(sorted2020))

199 valid queries. --> average: 0.0004992586004082329
0.0026061594538377055


Working with true labels instead of LTR output:

In [34]:
objective='EEL'
objective_args = {'grade_levels':2}

exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.tedlr).max()+2)])
y_rerank2020, sorted2020, min_vals = learn_all_query(  ds2020.telv, ds2020.teg, ds2020.tedlr, 
                                            exposure = exposure2020,
                                            epochs=5, lr=0.3, 
                                            learner_cls=permutationgraph.QueryLearner,
                                            objective=objective,
                                            objective_args = objective_args)
print(f'{len(min_vals[min_vals>0])} valid queries. --> average: {min_vals[min_vals>0].mean()}')

if objective == 'DTR':
    print(DTR.ndcg_dtr(exposure2020, ds2020.telv, y_rerank2020, ds2020.tedlr, ds2020.teg, ds2020.query_seq))
elif objective == 'EEL':
    eel = EEL.EEL(y_pred=ds2020.telv, g=ds2020.teg, dlr=ds2020.tedlr, exposure=exposure2020, grade_levels=2)
    print(eel.eval(sorted2020))

196 valid queries. --> average: 0.00540273725185993
9.125137616866798e-05


In [40]:
def test(epochs, lr):
    exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.tedlr).max()+2)])
    y_rerank2020, sorted2020, min_vals = learn_all_query(  ds2020.telv, ds2020.teg, ds2020.tedlr, 
                                            exposure = exposure2020,
                                            epochs=epochs, lr=lr, 
                                            learner_cls=permutationgraph.QueryLearner,
                                            objective=objective,
                                            objective_args = {'grade_levels':2})
    eel = EEL.EEL(y_pred=ds2020.telv, g=ds2020.teg, dlr=ds2020.tedlr, exposure=exposure2020, grade_levels=2)
    return eel.eval(sorted2020)

for epochs in [2,5,10,20,40,100]:
    print(epochs)
    for lr in [0,0.05,0.1,0.3]:
        eel = []
        for i in range(8):
            eel.append(test(5,lr))
        eel = np.array(eel)
        print([lr, eel.mean(), eel.std()])

2
[0, 3.4903578476035624e-06, 4.089276234288525e-06]
[0.05, 1.8794541399067257e-05, 3.24800807005403e-05]
[0.1, 1.7222257762569392e-05, 1.8957449224953783e-05]
[0.3, 1.75412902248232e-05, 1.227281485575612e-05]
5
[0, 1.2160567642181366e-05, 8.629643901219004e-06]
[0.05, 1.1491217244884844e-05, 1.3099901826197362e-05]
[0.1, 1.3804670305915144e-05, 2.5698098945966977e-05]
[0.3, 1.94569482926411e-05, 1.8635912386027702e-05]
10
[0, 3.582122066188701e-06, 3.2353885845058875e-06]
[0.05, 8.798772188069718e-06, 1.3343988574667018e-05]
[0.1, 3.877328137464268e-06, 6.882999492179967e-06]
[0.3, 9.384055295593371e-06, 9.52857559334133e-06]
20
[0, 2.6310944979426724e-05, 2.783596015671693e-05]
[0.05, 1.6972512263115772e-05, 1.8209822370835188e-05]
[0.1, 6.275123822347227e-06, 4.893906194199283e-06]
[0.3, 1.3698044535756684e-05, 1.1604093287587913e-05]
40
[0, 4.475516088413762e-06, 4.505769605885606e-06]
[0.05, 1.9392511266385842e-05, 1.2757770422882835e-05]
[0.1, 1.9519859634207754e-05, 1.744046988

In [36]:

for lr in [0,0.1,0.3,0.5]:
    eel = []
    for i in range(8):
        eel.append(test(5,lr))
    eel = np.array(eel)
    print([lr, eel.mean(), eel.std()])

TypeError: test() takes 1 positional argument but 2 were given

**TREC 2019**

In [20]:
exposure2019 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2019.tedlr).max()+2)])
y_rerank2019, min_vals = learn_all_query(  y_pred2019, ds2019.teg, ds2019.tedlr, 
                                            exposure=exposure2019,
                                            epochs=50, lr=0.3, 
                                            learner_cls=permutationgraph.QueryLearner)
print(f'{len(min_vals[min_vals>0])} valid queries.')
DTR.ndcg_dtr(exposure2019, ds2019.telv, y_rerank2019, ds2019.tedlr, ds2019.teg, ds2019.query_seq)

554 valid queries.


{'ndcg@5': 0.6973146692423147,
 'ndcg@10': 0.7554091354844698,
 'seq DTR': 0.9820242978616949,
 'single session DTR': 0.9865777804990237}

Some checks for weirdness!

In [22]:
print(DTR.ndcg_dtr(exposure2019, ds2019.telv, -y_rerank2019, ds2019.tedlr, ds2019.teg, ds2019.query_seq))
print(DTR.ndcg_dtr(exposure2019, ds2019.telv, y_pred2019, ds2019.tedlr, ds2019.teg, ds2019.query_seq))
print(DTR.ndcg_dtr(exposure2019, ds2019.telv, -y_pred2019, ds2019.tedlr, ds2019.teg, ds2019.query_seq))

{'ndcg@5': 0.6947353794524712, 'ndcg@10': 0.769091241344222, 'seq DTR': 0.9569590661138251, 'single session DTR': 0.9981156995633252}
{'ndcg@5': 0.7553542778927196, 'ndcg@10': 0.8181533543976589, 'seq DTR': 0.99412071452734, 'single session DTR': 0.9849812607892133}
{'ndcg@5': 0.6517401169984236, 'ndcg@10': 0.7206210810470921, 'seq DTR': 0.9864749597040254, 'single session DTR': 0.9927482857791988}


---


# Batch Learner

Test `BatchLearner` for a set of queries.

In [ ]:
def select_queries(y, g, dlr, qids):
    ys, gs, dlrs = [], [], [0]
    for qid in qids:
        s, e = dlr[qid:qid+2]
        ys.append(y[s:e])
        gs.append(g[s:e])
        dlrs.append(e-s)
    return np.concatenate(ys), np.concatenate(gs), np.cumsum(dlrs)

qids = [197,  64]
y_pred, gs, dlrs = select_queries(ds2020.telv, ds2020.teg, ds2020.tedlr, qids)
objective_ins = DTR.DTR(y_pred = y_pred, g = gs, dlr = dlrs, exposure = np.array([1./np.log2(2+i) for i in range(1,1000)]), method='batch_ratio')

ss = []
for qid in range(dlrs.shape[0] - 1):
    s,e = dlrs[qid:qid+2]
    ss.append(y_pred[s:e].argsort()[::-1])
sorted_docs = np.concatenate(ss)
batch_numbers = np.repeat(dlrs[:-1], np.diff(dlrs))
learner = permutationgraph.BatchLearner(objective_ins=objective_ins, sorted_docs=sorted_docs, intra=gs, inter=batch_numbers)
learner.fit(50, 0.3, True)

Test `BatchLearner` class for all queries, using `learn_all_batch` function.

In [34]:

def get_group_counts(g, dlr):
    groups = np.unique(g)
    gcnt = [[] for _ in range(len(groups))]
    for qid in range(dlr.shape[0] - 1):
        s, e = dlr[qid:qid+2]
        for i, group in enumerate(groups):
            gcnt[i].append(len(np.where(g[s:e] == group)[0]))
    for i, group in enumerate(groups):
            gcnt[i] = np.array(gcnt[i])
    return groups, gcnt


def update_y(y, sorted_docs, dlr, qids):
    pos = 0
    for qid in qids:
        s, e = dlr[qid:qid+2]
        scores = np.arange(e-s, 0, -1)
        y[s:e] = scores[sorted_docs[pos:pos+e-s]]
        pos += e-s
def learn_all_batch(y_pred, g, dlr, exposure, epochs, lr, learner_cls):
    y_rerank = []
    min_vals = []
    
    groups, gcnt = get_group_counts(g, dlr)
    for i, _ in enumerate(groups):
            gcnt[i] = gcnt[i].argsort()[:, None]
    gcnt = np.concatenate(gcnt, axis=1)
    
    for qid in range(dlr.shape[0] - 1):
        s, e = dlr[qid:qid+2]

        sorted_docs = y_pred[s:e].argsort()[::-1]

        scores = np.arange(len(sorted_docs), 0, -1)
        y_rerank.append(scores[sorted_docs])
    y_rerank = np.concatenate(y_rerank)

    # for bid in trange(dlr.shape[0] - 1, leave=False):
    for bid in range(dlr.shape[0] - 1):
        qids = gcnt[bid, :]
        # print(qids)

        ys, gs, dlrs = select_queries(y_rerank, g, dlr, qids)


        objective_ins = DTR.DTR(y_pred = ys, g = gs, dlr = dlrs, exposure = exposure, method='batch_ratio')

        ss = []
        for qid in range(dlrs.shape[0] - 1):
            s,e = dlrs[qid:qid+2]
            ss.append(ys[s:e].argsort()[::-1])
        sorted_docs = np.concatenate(ss)
        batch_numbers = np.repeat(dlrs[:-1], np.diff(dlrs))
        learner = learner_cls(objective_ins=objective_ins, sorted_docs=sorted_docs, intra=gs, inter=batch_numbers)

        vals = learner.fit(epochs, lr, verbose=False)

        update_y(y_rerank, learner.sorted_docs, dlr, qids)
        
        vals = np.array(vals)
        min_vals.append(vals.min())

        
    return y_rerank, np.array(min_vals)
   

**TREC 2020**

In [35]:

exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.tedlr).max()+2)])
y_rerank2020, min_vals = learn_all_batch(  y_pred2020, ds2020.teg, ds2020.tedlr, 
                                            exposure = exposure2020,
                                            epochs=200, lr=0.3, 
                                            learner_cls=permutationgraph.BatchLearner)
print(f'{len(min_vals[min_vals>0])} valid queries. --> average: {min_vals[min_vals>0].mean()}')

DTR.ndcg_dtr(exposure2020, ds2020.telv, y_rerank2020, ds2020.tedlr, ds2020.teg, ds2020.query_seq)


200 valid queries. --> average: 1.08220164538563


{'ndcg@5': 0.35520380092632026,
 'ndcg@10': 0.3970282016641636,
 'seq DTR': 0.30804457586821893,
 'single session DTR': 0.7601551338899333}

Testing with true labels:

In [36]:

exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.tedlr).max()+2)])
y_rerank2020, min_vals = learn_all_batch(  ds2020.telv, ds2020.teg, ds2020.tedlr, 
                                            exposure = exposure2020,
                                            epochs=200, lr=0.3, 
                                            learner_cls=permutationgraph.BatchLearner)
print(f'{len(min_vals[min_vals>0])} valid queries. --> average: {min_vals[min_vals>0].mean()}')

DTR.ndcg_dtr(exposure2020, ds2020.telv, y_rerank2020, ds2020.tedlr, ds2020.teg, ds2020.query_seq)

200 valid queries. --> average: 1.092158462578101


{'ndcg@5': 0.25129779144086084,
 'ndcg@10': 0.2687497832061068,
 'seq DTR': 0.30714529671008617,
 'single session DTR': 0.7686791385516936}

---

# Nonrelevant Tests

In [ ]:
df = pd.DataFrame({'qid':list(np.repeat(ds2019.teqid, np.diff(ds2019.tedlr))), 'group':list(ds2019.teg), 'label':list(ds2019.telv), 'pred':list(y_pred2019)})

In [ ]:
n = 4
learner = permutationgraph.QueryLearner(np.zeros(4), np.arange(n), None, sorted_docs = None)
learner.probs_mat = 0.5 * np.ones([learner.n, learner.n])

def per2int(docs):
    return (np.array([10**(len(docs)-1-i) for i in range(len(docs))])*docs).sum()
freq = defaultdict(lambda:0)
iters = 50000
for _ in range(iters):
    docs, crap = learner.permute()
    freq[per2int(docs)] += 2.**(n+1)/iters
a = sorted(freq.items(),key=lambda x: x[1])
a

In [12]:

def linspan(y_pred, levels):
    m,M = y_pred.min()-1e-10, y_pred.max()+1e-10
    m = max(m,0)
    step = (M - m) / levels
    return np.floor((y_pred-m)/step)


def disc_target_exposure(y, exposure):
    sorted_y = np.sort(y)[::-1]
    expo = exposure[:len(y)]
    te = []
    for g in range(int(y.max()+1)):
        te.append(np.mean(expo[sorted_y==g]))
    return np.array(te)
y_pred = np.random.rand(20)


exposure = np.array([1./np.log2(2+i) for i in range(1,1000+2)])

print(y_pred)
a = linspan(y_pred, 5)
b = disc_target_exposure(linspan(y_pred, 5), exposure)
[a,b[a.astype(int)]]


[0.22604073 0.70783217 0.43035958 0.51833013 0.20620828 0.45995174
 0.48296419 0.32335147 0.25798429 0.49642818 0.90861669 0.06843116
 0.50280473 0.18948366 0.629371   0.74419906 0.55278362 0.63990797
 0.16675551 0.45046706]


[array([0., 3., 2., 2., 0., 2., 2., 1., 1., 2., 4., 0., 2., 0., 3., 4., 2.,
        3., 0., 2.]),
 array([0.23170273, 0.39124552, 0.28833521, 0.28833521, 0.23170273,
        0.28833521, 0.28833521, 0.24732527, 0.24732527, 0.28833521,
        0.56546488, 0.23170273, 0.28833521, 0.23170273, 0.39124552,
        0.56546488, 0.28833521, 0.39124552, 0.23170273, 0.28833521])]

In [82]:

def get_edges(args):
    edges = 0
    for i in range(len(args)):
        for j in range(i+1, len(args)):
            if args[i] > args[j]:
                edges += 1
    return edges

def sample(PPG):
    n = PPG.shape[0]
    if n <= 1:
        return np.arange(n)
    selected = np.random.binomial(1,PPG)
    positions = np.arange(n) + selected.sum(1) - selected.sum(0)
    # print(positions)
    empty_positions = []
    for i in range(n):
        shared_i_s = np.where(positions == i)[0]
        if len(shared_i_s) <= 1:
            if len(shared_i_s) == 0:
                empty_positions.append(i)
            continue
        chosen_i = np.random.choice(shared_i_s)
        for j in shared_i_s:
            if j == chosen_i:
                continue
            positions[j] = -1
    remaining = np.where(positions == -1)[0]
    # print(remaining)
    if len(remaining) > 0:
        PPG2 = PPG[remaining,:][:,remaining]
        positions2 = sample(PPG2)
        positions[remaining] = np.array(empty_positions)[positions2]
    return positions


n = 5
prob = 0.3
PPG = prob * np.triu(np.ones((n,n)), 1)

from collections import defaultdict
from tqdm.notebook import trange

freq = defaultdict(lambda:defaultdict(lambda:0))

iters = 100000
for i in trange(iters):
    positions = sample(PPG)
    edges = get_edges(positions)
    freq[edges][str(positions)] += 1./iters



  0%|          | 0/100000 [00:00<?, ?it/s]

In [83]:
for edge in freq:
    freq_ = freq[edge]

    vals = np.array(list(freq_.values()))
    p = (prob**edge)*((1.-prob)**((n*(n-1)/2)-edge))
    print([edge, len(vals), vals.mean()/p, vals.mean(), vals.std(), vals.min(), vals.max(), p])

[5, 22, 11.065081174757642, 0.004519090909090883, 0.0011899805540462327, 0.0025000000000000057, 0.0074299999999998655, 0.00040841009999999976]
[3, 15, 6.18615895130482, 0.013755333333332941, 0.0029847811012235887, 0.009429999999999784, 0.019789999999999364, 0.002223566099999999]
[1, 4, 3.2543559241186224, 0.03939750000000139, 0.0011521149031241689, 0.03775000000000089, 0.040810000000001824, 0.012106082099999993]
[0, 1, 2.1832001288016656, 0.06167000000000821, 0.0, 0.06167000000000821, 0.06167000000000821, 0.02824752489999998]
[4, 20, 8.183476083755576, 0.0077984999999998516, 0.0025695160536566924, 0.0047799999999999735, 0.013869999999999603, 0.0009529568999999997]
[7, 15, 27.381518941105774, 0.0020540000000000016, 0.0011889367799284604, 0.0008300000000000015, 0.004689999999999977, 7.501409999999996e-05]
[2, 9, 4.512057070332568, 0.023409999999999216, 0.0034555784207888044, 0.019089999999999392, 0.029789999999998956, 0.005188320899999997]
[6, 20, 16.788272376221837, 0.002938499999999998

In [86]:
sorted(list(freq[7].items()), key=lambda x: x[1])

[('[3 2 4 0 1]', 0.0008300000000000015),
 ('[3 4 0 2 1]', 0.0008900000000000017),
 ('[3 4 1 0 2]', 0.0012000000000000025),
 ('[2 4 3 0 1]', 0.0012000000000000025),
 ('[2 3 4 1 0]', 0.0012300000000000026),
 ('[4 3 0 1 2]', 0.0012600000000000027),
 ('[4 0 3 2 1]', 0.0015100000000000033),
 ('[3 2 1 4 0]', 0.0017000000000000038),
 ('[3 1 4 2 0]', 0.0017200000000000039),
 ('[4 2 0 3 1]', 0.0017200000000000039),
 ('[4 1 3 0 2]', 0.0024000000000000054),
 ('[2 4 1 3 0]', 0.0024800000000000056),
 ('[4 2 1 0 3]', 0.003680000000000009),
 ('[1 4 3 2 0]', 0.004299999999999993),
 ('[4 1 2 3 0]', 0.004689999999999977)]

In [81]:
import math
def ratio(n):
    print([n, np.sqrt(((n*(n-1)/2.) - np.log2(math.factorial(n)))*2)])

for i in range(5,20):
    ratio(i)

[5, 2.4872110503097566]
[6, 3.319080265275405]
[7, 4.171520581661493]
[8, 5.0399984090498915]
[9, 5.921294956370677]
[10, 6.813066693535745]
[11, 7.71356043168727]
[12, 8.621431988472054]
[13, 9.53562845834332]
[14, 10.455309667888933]
[15, 11.379794332949729]
[16, 12.308522212688034]
[17, 13.241026900423323]
[18, 14.176915862586243]
[19, 15.115855528151963]


In [86]:
def _neighbors(mat):
    neigh = {'upper':{}, 'lower':{}}
    n = mat.shape[0]
    for i in range(n):
        neigh['upper'][i] = []
        for j in range(i):
            if mat[j,i] == 1:
                neigh['upper'][i].append(j)
    for i in range(n):
        neigh['lower'][i] = []
        for j in range(i+1,n):
            if mat[i,j] == 1:
                neigh['lower'][i].append(j)
    return neigh

def _insert_to_down(merged, PPG, i_u, up):
    Nu = PPG.shape[0]
    # print('inserting index', i_u)
    # print('merged:', merged)
    # print('PPG:', PPG)

    if i_u < up.shape[0] - 1:
        after_ind = int(np.where(merged == up[i_u + 1])[0])
    else:
        after_ind = merged.shape[0]

    if after_ind == i_u + 1:
        # print('no space to move')
        return

    for i_d in range(i_u+1, after_ind):
        q_u, q_d = 0, 0
        
        for k in range(i_d+1, after_ind):
            q_d = q_d * (1. - PPG[merged[i_u]][merged[k]]) + PPG[merged[i_u]][merged[k]]

        for k in range(i_u):
            q_u = q_u * (1. - PPG[merged[k]][merged[i_d]]) + PPG[merged[k]][merged[i_d]]

        q = q_u + q_d - (q_u * q_d)
        q *= 1. - PPG[merged[i_u]][merged[i_d]]
        if np.random.binomial(1, PPG[merged[i_u]][merged[i_d]] / (1. - q)) == 0:
            break

    # print('q_u:', q_u, 'q_d:', q_d, 'q:', q, 'p:', PPG[i_u][i_d])
    if i_d > i_u + 1:
        shift = merged[i_u+1:i_d]
        merged_i_u = merged[i_u]
        merged[i_u:i_d-1] = shift
        merged[i_d-1] = merged_i_u

    
def get_permutation(selected):
    return np.arange(selected.shape[0]) + selected.sum(1) - selected.sum(0)

def PPG_merge(up, down, PPG):
    Nu = up.shape[0]
    Nd = down.shape[0]
    
    down += Nu
    merged = np.concatenate([up, down])
    # print('merge -> up:', up)
    # print('down:', down)
    # print('PPG:', PPG)

    for i_u in reversed(range(Nu)):
        _insert_to_down(merged, PPG, i_u, up)
    return merged

def PPG_sample(PPG):
    n = PPG.shape[0]
    mid = n // 2
    # print('main:', n, mid)
    if n == 1:
        return np.array([0])
    if n == 2:
        if np.random.binomial(1,PPG[0,1]):
            return np.array([1,0])
        return np.array([0,1])
    up = PPG_sample(PPG[:mid,:][:,:mid])
    down = PPG_sample(PPG[mid:,:][:,mid:])
    mat = PPG_merge(up, down, PPG)
    # print('PPG:', PPG)
    # print('mat:', mat)
    return mat



In [93]:

n = 5
prob = 0.5 * np.random.rand(n,n)
PPG = prob * np.triu(np.ones((n,n)), 1)

from collections import defaultdict
from tqdm.notebook import trange


def get_edges(args):
    edges = 0
    for i in range(len(args)):
        for j in range(i+1, len(args)):
            if args[i] > args[j]:
                edges += 1
    return edges
    
freq = defaultdict(lambda:defaultdict(lambda:0))

iters = 1000
for i in trange(iters):
    positions = PPG_sample(PPG)
    edges = get_edges(positions)
    freq[edges][str(positions)] += 1./iters
# print(positions)

TypeError: 'list' object cannot be interpreted as an integer

In [92]:
for edge in freq:
    freq_ = freq[edge]

    vals = np.array(list(freq_.values()))
    p = (prob**edge)*((1.-prob)**((n*(n-1)/2)-edge))
    print('edges:', edge, 'count:', len(vals), 'ratio:', vals.mean()/p, 'max to min:', vals.max()/vals.min())

for edge in freq:
    freq_ = list(freq[edge].items())
    print(freq_)

edges: 5 count: 4 ratio: 61.44000000000004 max to min: 3.3333333333333335
edges: 4 count: 7 ratio: 42.7154285714286 max to min: 4.105263157894737
edges: 3 count: 8 ratio: 26.624000000000017 max to min: 3.7142857142857157
edges: 2 count: 7 ratio: 18.578285714285727 max to min: 3.888888888888891
edges: 1 count: 4 ratio: 13.312000000000005 max to min: 1.6666666666666672
edges: 6 count: 1 ratio: 78.84800000000006 max to min: 1.0
edges: 0 count: 1 ratio: 4.096 max to min: 1.0
[('[4 0 2 1 3]', 0.08100000000000006), ('[2 1 4 0 3]', 0.04200000000000003), ('[3 1 2 0 4]', 0.09000000000000007), ('[1 4 2 0 3]', 0.027000000000000017)]
[('[0 4 2 1 3]', 0.03300000000000002), ('[3 0 2 1 4]', 0.07800000000000006), ('[2 0 4 1 3]', 0.04100000000000003), ('[1 4 0 2 3]', 0.03100000000000002), ('[1 3 2 0 4]', 0.046000000000000034), ('[2 1 3 0 4]', 0.04400000000000003), ('[1 2 4 0 3]', 0.01900000000000001)]
[('[0 4 1 2 3]', 0.03000000000000002), ('[1 2 0 4 3]', 0.014000000000000005), ('[1 0 4 2 3]', 0.019000

In [11]:
import numpy as np
import torch

x = np.array([[1,2,3],[4,5,6]])
y = np.array([[0,2,1],[2,1,0]])
x = torch.FloatTensor(x)
y = torch.LongTensor(y)
x[torch.arange(x.shape[0]).unsqueeze(1).repeat((1,3)).flatten(), y.flatten()].view(2,3)

tensor([[1., 3., 2.],
        [6., 5., 4.]])

In [19]:
from itertools import permutations
list(permutations(range(4)))

[(0, 1, 2, 3),
 (0, 1, 3, 2),
 (0, 2, 1, 3),
 (0, 2, 3, 1),
 (0, 3, 1, 2),
 (0, 3, 2, 1),
 (1, 0, 2, 3),
 (1, 0, 3, 2),
 (1, 2, 0, 3),
 (1, 2, 3, 0),
 (1, 3, 0, 2),
 (1, 3, 2, 0),
 (2, 0, 1, 3),
 (2, 0, 3, 1),
 (2, 1, 0, 3),
 (2, 1, 3, 0),
 (2, 3, 0, 1),
 (2, 3, 1, 0),
 (3, 0, 1, 2),
 (3, 0, 2, 1),
 (3, 1, 0, 2),
 (3, 1, 2, 0),
 (3, 2, 0, 1),
 (3, 2, 1, 0)]